In [ ]:

gmx_mpi pdb2gmx -f ../template.pdb -o template.gro -water tip3p

# -------------------------------------------------------
# Program:     gmx pdb2gmx, version 2022.5-plumed_2.9.0
# Source file: src/gromacs/gmxpreprocess/ter_db.cpp (line 138)
# Function:    void read_atom(char*, bool, std::string*, t_atom*, PreprocessingAtomTypes*, int*)

# Inconsistency in user input:
# Atom type OH specified in terminal database has not been defined in the force
# field

# For more information and tips for troubleshooting, please check the GROMACS
# website at http://www.gromacs.org/Documentation/Errors
# -------------------------------------------------------

# Changed aminoacids.c.tdb OH to OH1.This worked! Potential typo in force field files?

gmx_mpi editconf -f template.gro -o template_newbox.gro -c -d 1.0 -bt dodecahedron
# output of the above command Volume: 362.038 nm^3

gmx_mpi solvate -cp template_newbox.gro -cs spc216.gro -o template_solv.gro -p ../topol.top
# only 5922 solvent molecules added compared to 11000 in the paper

# WARNING: Masses and atomic (Van der Waals) radii will be guessed
#          based on residue and atom names, since they could not be
#          definitively assigned from the information in your input
#          files. These guessed numbers might deviate from the mass
#          and radius of the atom type. Please check the output
#          files if necessary. Note, that this functionality may
#          be removed in a future GROMACS version. Please, consider
#          using another file format for your input.

# NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
# from the source below. This means the results may be different
# compared to previous GROMACS versions.

# ++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
# A. Bondi
# van der Waals Volumes and Radii
# J. Phys. Chem. 68 (1964) pp. 441-451

In [2]:
# adding NaCl and KCl to the box

# am confused about the mdp file used. the file uses 1nm as the cutoff distances for WDW/coulomb interactions
# while the cutoff used in actual simulation is 1.2
gmx_mpi grompp -f ../mdp_files/ions.mdp -c template_solv.gro -p ../topol.top -o genion_NaCl_input.tpr
gmx_mpi genion -s genion_NaCl_input.tpr -o genion_NaCl_output.gro -conc 0.137 -pname NA -nname CL -p ../topol.top

gmx_mpi grompp -f ../mdp_files/ions.mdp -c genion_NaCl_output.gro -p ../topol.top -o genion_KCl_input.tpr
gmx_mpi genion -s genion_KCl_input.tpr -o genion_KCl_output.gro -conc 0.00268 -pname K -nname CL -p ../topol.top

In [4]:
# energy minimization
# same confusion as previous cell regarding mdp file, cutoff distances and other parameters different
gmx_mpi grompp -f ../mdp_files/em.mdp -c genion_KCl_output.gro -p ../topol.top -o em_input.tpr -maxwarn 1
gerun gmx_mpi mdrun -s em_input.tpr -o em.trr -x em.xtc -c em_output.gro -g em.log -e em.edr -v -maxh 10

# Steepest Descents converged to Fmax < 1000 in 101 steps
# Potential Energy  = -2.6635816e+05
# Maximum force     =  9.3316010e+02 on atom 409
# Norm of force     =  7.3510789e+01

In [ ]:
# preliminary 2-ns simulation at 600 K in the NVT ensemble
# the paper doesn't mention position restraints. so no position restraints? obv no posres cuz want to find max rms replicas
# also which thermostat to use?
gmx_mpi grompp -f ../mdp_files/nvt-run-600.mdp -c em_output.gro -p ../topol.top -o nvt_600_input.tpr -maxwarn 1
gerun gmx_mpi mdrun -s nvt_600_input.tpr -o nvt_600.trr -x nvt_600.xtc -c nvt_600_output.gro -g nvt_600.log -e nvt.edr -v &> nvt_600.log

In [ ]:
# algorithm to extract 48 confirmations

In [ ]:
# NVT and NPT equilibration with position restraints on heavy atoms
